# ETL para Redis

## Instalações

In [2]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


## Importações

In [3]:
import os
import sys
from tqdm import tqdm

from pyspark.sql import SparkSession

## Constantes

In [4]:
DATASET_PATH = '../../dataset'

USER_PATH = os.path.join(DATASET_PATH, 'users-details-2023.csv')
CONSUMED_ANIMES_PATH = os.path.join(DATASET_PATH, 'final_animedataset.csv')

## Código para resolver um problema de versão encontrada pelo PySpark

A resolução do erro foi encontrada em uma resposta no [StackOverflow](https://stackoverflow.com/a/65010346)

In [5]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

## Configurando inicialização do "conector" Neo4j spark

In [6]:
spark = SparkSession.\
        builder.\
        config("spark.jars", "../../spark-redis/target/spark-redis_2.12-3.1.0-SNAPSHOT-jar-with-dependencies.jar").\
        getOrCreate()

## Lendo o arquivo users-details-2023.csv

In [7]:
df_user = spark.read.csv(USER_PATH, inferSchema=True, header=True, escape='"', multiLine=True)
df_user = df_user.select(['mal ID', 'username', 'gender', 'days watched'])

## Lendo o arquivo final_animedataset.csv

In [8]:
df_consumed_animes = spark.read.csv(CONSUMED_ANIMES_PATH, inferSchema=True, header=True, escape='"', multiLine=True)

## Gerando os agregados

### Coletando as recomendações para os usuários

### Criando os agregados iniciais

In [12]:
agregados = {}

users_pandas = df_user.toPandas()
qnt_users = len(users_pandas)

for _, user in tqdm(users_pandas.iterrows(), total=len(users)):
    user = users[i]
    user_id = user['mal ID']
    
    agregados[user_id] = {}
    agregados[user_id]['username'] = user['username']
    agregados[user_id]['gender'] = user['gender']
    agregados[user_id]['joined'] = user['joined']
    agregados[user_id]['days watched'] = user['days watched']
    if user_id in recomendations:
        agregados[user_id]['recomendations'] = recomendations[user_id]

OSError: [Errno 22] Invalid argument

### Adicionando aos agregados os animes que os usuários assistiram

### Tranformando agregados de dicionário para DataFrame Spark

### Salvando os dados no Redis